In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor


import matplotlib.pyplot as plt

print(torch.__version__)
print(torchvision.__version__)

2.3.1
0.18.1


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_data = datasets.CIFAR100(
    root="data",
    train=True,
    transform=ToTensor(),
    download=True,
    target_transform=None
    )


test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)



Files already downloaded and verified
Files already downloaded and verified


In [4]:
from torch.utils.data import DataLoader


In [5]:
BATCH_SIZE = 32
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    shuffle=True)






In [6]:
train_data.class_to_idx

{'apple': 0,
 'aquarium_fish': 1,
 'baby': 2,
 'bear': 3,
 'beaver': 4,
 'bed': 5,
 'bee': 6,
 'beetle': 7,
 'bicycle': 8,
 'bottle': 9,
 'bowl': 10,
 'boy': 11,
 'bridge': 12,
 'bus': 13,
 'butterfly': 14,
 'camel': 15,
 'can': 16,
 'castle': 17,
 'caterpillar': 18,
 'cattle': 19,
 'chair': 20,
 'chimpanzee': 21,
 'clock': 22,
 'cloud': 23,
 'cockroach': 24,
 'couch': 25,
 'crab': 26,
 'crocodile': 27,
 'cup': 28,
 'dinosaur': 29,
 'dolphin': 30,
 'elephant': 31,
 'flatfish': 32,
 'forest': 33,
 'fox': 34,
 'girl': 35,
 'hamster': 36,
 'house': 37,
 'kangaroo': 38,
 'keyboard': 39,
 'lamp': 40,
 'lawn_mower': 41,
 'leopard': 42,
 'lion': 43,
 'lizard': 44,
 'lobster': 45,
 'man': 46,
 'maple_tree': 47,
 'motorcycle': 48,
 'mountain': 49,
 'mouse': 50,
 'mushroom': 51,
 'oak_tree': 52,
 'orange': 53,
 'orchid': 54,
 'otter': 55,
 'palm_tree': 56,
 'pear': 57,
 'pickup_truck': 58,
 'pine_tree': 59,
 'plain': 60,
 'plate': 61,
 'poppy': 62,
 'porcupine': 63,
 'possum': 64,
 'rabbit': 65,

In [7]:
class_names=train_data.classes
class_names

['apple',
 'aquarium_fish',
 'baby',
 'bear',
 'beaver',
 'bed',
 'bee',
 'beetle',
 'bicycle',
 'bottle',
 'bowl',
 'boy',
 'bridge',
 'bus',
 'butterfly',
 'camel',
 'can',
 'castle',
 'caterpillar',
 'cattle',
 'chair',
 'chimpanzee',
 'clock',
 'cloud',
 'cockroach',
 'couch',
 'crab',
 'crocodile',
 'cup',
 'dinosaur',
 'dolphin',
 'elephant',
 'flatfish',
 'forest',
 'fox',
 'girl',
 'hamster',
 'house',
 'kangaroo',
 'keyboard',
 'lamp',
 'lawn_mower',
 'leopard',
 'lion',
 'lizard',
 'lobster',
 'man',
 'maple_tree',
 'motorcycle',
 'mountain',
 'mouse',
 'mushroom',
 'oak_tree',
 'orange',
 'orchid',
 'otter',
 'palm_tree',
 'pear',
 'pickup_truck',
 'pine_tree',
 'plain',
 'plate',
 'poppy',
 'porcupine',
 'possum',
 'rabbit',
 'raccoon',
 'ray',
 'road',
 'rocket',
 'rose',
 'sea',
 'seal',
 'shark',
 'shrew',
 'skunk',
 'skyscraper',
 'snail',
 'snake',
 'spider',
 'squirrel',
 'streetcar',
 'sunflower',
 'sweet_pepper',
 'table',
 'tank',
 'telephone',
 'television',
 'tig

In [8]:
train_features_batch , train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape,train_labels_batch.shape

(torch.Size([32, 3, 32, 32]), torch.Size([32]))

In [9]:
torch.cuda.manual_seed(42)


class CNN(nn.Module):
    def __init__(self,input,hidden,output):
        super().__init__()
        self.cnn_block1= nn.Sequential(
            nn.Conv2d(in_channels=input,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            )
        self.cnn_block2= nn.Sequential(
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            )
        self.cnn_block3= nn.Sequential(
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden*8*8,out_features=output)
        )

    def forward(self,x):
        x = self.cnn_block1(x)
        x= self.cnn_block2(x)
        # x= self.cnn_block3(x)
        # print(x.shape)
        x=self.classifier(x)

        return x 

In [10]:
Model = CNN(input=3,hidden=10,output=len(class_names)).to(device)

In [11]:
image,label = test_data[0]

In [12]:
image.shape

torch.Size([3, 32, 32])

In [13]:
Model(image.to(device).unsqueeze(0))

tensor([[ 0.0147, -0.0284,  0.0306, -0.0294, -0.0798,  0.0092,  0.0072,  0.0697,
          0.0395, -0.0047,  0.0226,  0.0172, -0.0033,  0.0577, -0.0144,  0.0567,
         -0.0322,  0.0707, -0.0110,  0.0284,  0.0738, -0.0337,  0.0206,  0.0064,
          0.1304,  0.0055,  0.0245,  0.0963,  0.0809,  0.0178,  0.0062,  0.0573,
         -0.0095,  0.0048,  0.0209, -0.0435, -0.0176,  0.0126,  0.0264, -0.0054,
         -0.0338, -0.0133,  0.0903,  0.0192, -0.0069, -0.0131,  0.0457, -0.0106,
         -0.0558, -0.0274,  0.0232,  0.0107, -0.0381,  0.0078, -0.0719, -0.0085,
         -0.0191,  0.0338,  0.0467,  0.1078, -0.0057,  0.0477,  0.0143,  0.0104,
          0.0126, -0.0016,  0.0133, -0.0388, -0.0468,  0.0619,  0.0450, -0.0419,
          0.0470,  0.0025, -0.0041, -0.0147,  0.0351, -0.0063, -0.0194,  0.0951,
          0.0278, -0.0385,  0.0070, -0.0296,  0.0129,  0.0771,  0.0043,  0.0085,
         -0.0739,  0.0207, -0.0484,  0.0012,  0.0389, -0.0511,  0.0053,  0.1055,
         -0.0026,  0.0517,  

In [15]:
from tqdm.auto import tqdm

In [16]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=Model.parameters(),
                            lr=0.1)

In [17]:
from helper_functions import accuracy_fn

In [18]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)


epochs = 4

for epoch in tqdm(range(epochs)):

    print(f"epoch: {epoch}-------")

    train_loss , train_acc = 0,0

    

    for batch,(X,Y) in enumerate(train_dataloader):
        Model.train()

        X,Y=X.to(device),Y.to(device)

        y_pres = Model(X)

        loss = loss_fn(y_pres,Y)

        train_loss += loss
        train_acc += accuracy_fn(y_true=Y,
                                 y_pred=y_pres.argmax(dim=1))

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()


    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)

    print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")




    test_loss , test_acc = 0 , 0
    Model.eval()

    with torch.inference_mode():
        for X_test ,Y_test in test_dataloader:

            X_test,Y_test=X.to(device),Y.to(device)

            

            # forward pass
            test_pred = Model(X_test)

            # calculate loss (accumulatively)
            test_loss += loss_fn(test_pred, Y_test)

            # Accuracy
            test_acc += accuracy_fn(Y_test,test_pred.argmax(dim=1))

        # Calculate the test loss average per batch
        test_loss /= len(test_dataloader)

        # Calculate the test acc average per batch

        test_acc /= len(test_dataloader)

        print(f"Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")





  0%|          | 0/4 [00:00<?, ?it/s]

epoch: 0-------
Train loss: 4.36494 | Train acc: 4.33%


 25%|██▌       | 1/4 [00:12<00:38, 12.75s/it]

Test loss: 2.32912 | Test acc: 56.25%

epoch: 1-------
Train loss: 3.71833 | Train acc: 14.64%


 50%|█████     | 2/4 [00:25<00:25, 12.64s/it]

Test loss: 2.42208 | Test acc: 37.50%

epoch: 2-------
Train loss: 3.36777 | Train acc: 20.82%


 75%|███████▌  | 3/4 [00:38<00:12, 12.78s/it]

Test loss: 3.01123 | Test acc: 31.25%

epoch: 3-------
Train loss: 3.17550 | Train acc: 24.39%


100%|██████████| 4/4 [00:51<00:00, 12.79s/it]

Test loss: 1.58381 | Test acc: 68.75%



In [ ]:
#Create a CNN

class FashionMNISTModelV2(nn.Module):

    """
    Model architecture that replicates the TinyVGG
    model from CNN explainer website.

    """

    def __init__(self,input:int,
                 hidden:int,
                 output:int):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden,out_channels=hidden,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden*8*8, # there's a trick to calculating this
                      out_features=output)
        )



    def forward(self,x):
        x=self.conv_block1(x)
        # print(x.shape)
        x=self.conv_block2(x)
        # print(x.shape)
        x=self.classifier(x)
        # print(x.shape)
        return x    

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input=3,hidden=10,output=len(class_names)).to(device)

In [ ]:
def train_step(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device = device):
    
    """Performs a training with model trying to learn on data_loader."""

    train_loss , train_acc = 0,0

    model.train()

    for batch,(X,Y) in enumerate(data_loader):
       

        X,Y=X.to(device),Y.to(device)
        # 1. Forward pass
        y_pred = model(X)

        # 2. Loss

        loss = loss_fn(y_pred,Y)
        train_loss +=loss # accumulate train loss
        train_acc += accuracy_fn(y_true=Y,
                                 y_pred=y_pred.argmax(dim=1))

        # 3. Optimizer zero grad

        optimizer.zero_grad()

        # 4. Loss backward

        loss.backward()


        # 5. optimizer step

        optimizer.step()



    train_loss /= len(data_loader)
    train_acc /= len(data_loader)

    print(f"Train loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

    


In [ ]:
def test_step(model:torch.nn.Module,
              data_loader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              accuracy_fn,
              device:torch.device=device):
    

    test_loss , test_acc = 0 , 0
    model.eval()

    with torch.inference_mode():
        for X ,Y in data_loader:
            X,Y = X.to(device),Y.to(device)
            # forward pass
            test_pred = model(X)

            # calculate loss (accumulatively)
            test_loss += loss_fn(test_pred, Y)

            # Accuracy
            test_acc += accuracy_fn(Y,test_pred.argmax(dim=1))

        # Calculate the test loss average per batch
        test_loss /= len(data_loader)

        # Calculate the test acc average per batch

        test_acc /= len(data_loader)

        print(f"Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")


    
    

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
from timeit import default_timer as timer
train_strat = timer()

for epoch in tqdm(range(epochs)):
    train_step(model=model_2,
              data_loader=train_dataloader,
              loss_fn=loss_fn,
              optimizer=optimizer,
              accuracy_fn=accuracy_fn,
              device=device)
    

    test_step(model=model_2,
              data_loader=test_dataloader,
              loss_fn=loss_fn,
              accuracy_fn=accuracy_fn,
              device=device)
    


train_end = timer()


In [19]:
def eval_model_gpu(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn):
    """Returns a dictionary containing the results of model predicting on data_loader."""
    loss,acc = 0,0
    model.eval()

    with torch.inference_mode():
        for X,Y in tqdm(data_loader):
            # Make prediction
            X,Y = X.to(device),Y.to(device)
            y_pred = model(X)

            # Accumulate the loss and acc values per batch
            loss += loss_fn(y_pred,Y)

            acc += accuracy_fn(Y,y_pred.argmax(dim=1))
            
        
        # Scale loss and acc to find the average loss/acc per batch

        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"Model_name": model.__class__.__name__,
            "model_loss":loss.item(),
            "model_acc":acc}




In [20]:
model_result = eval_model_gpu(model=Model,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)

100%|██████████| 313/313 [00:01<00:00, 201.12it/s]


In [21]:
model_result


{'Model_name': 'CNN',
 'model_loss': 3.4409008026123047,
 'model_acc': 21.206070287539937}